In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import math

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import pandas as pd

##############################################################################

def filterinitialset (filename, headername = "vibrational level v\Temperature(K)",  \
    factor = 1.0, normalize = False):

    dfin = pd.read_excel(filename)

    #print(dfin)
    
    dfdict = {}
    
    tempvalues = list(dfin.columns[1:])
    vibvalues = list(dfin[headername].values)

    min = float("inf")
    max = float("-inf")
    for c in dfin.columns:
        dfdict[c] = []
        if c == headername:
            dfdict[c] = list(dfin[c].values)
        else:
            for v in dfin[c].values:
                val = factor*v
                if val > max:
                    max = val
                if val < min:
                    min = val

    for c in dfin.columns:
        if c != headername:
            for v in dfin[c].values:
                val = factor*v
                valp = (val - min) / (max - min)
                if normalize:
                    dfdict[c].append(valp)
                else:
                    dfdict[c].append(val)

    df = pd.DataFrame.from_dict(dfdict)

    return df, vibvalues, tempvalues

##############################################################################

x_data = np.linspace(-10, 10, num=1000)
y_data = 0.1*x_data*np.cos(x_data) + 0.1*np.random.normal(size=1000)
print('Data created successfully')

filename = "N2N2_dataset.xls"
df, vib_values , temp_values = filterinitialset (filename)
local_temp_values = temp_values

thefirst = True

maxt = max(local_temp_values)
mint = min(local_temp_values)

for vibidx in range(len(vib_values)-1, len(vib_values)):
    train_x = []
    train_y = []

    maxy = float("-inf")
    miny = float("+inf")

    for idx, t in enumerate(local_temp_values):
        yval = df[t].values[vibidx]

        if yval < miny:
            miny = yval
        elif yval > maxy:
            maxy = yval

    for idx, t in enumerate(local_temp_values):
        tval = (t - mint)/(maxt - mint)
        train_x.append(tval)
    
        yval = df[t].values[vibidx]
        train_y.append((yval - miny)/(maxy - miny))

    if thefirst:
        val_x = train_x.copy()
        val_y = train_y.copy()
        thefirst = False

    minx = min(train_x)
    maxx = max(train_x)

    train_x = np.asarray(train_x).reshape(-1, 1)
    train_y = np.asarray(train_y)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(units = 1, activation = 'linear', input_shape=[1]))
model.add(keras.layers.Dense(units = 64, activation = 'relu'))
model.add(keras.layers.Dense(units = 64, activation = 'relu'))
model.add(keras.layers.Dense(units = 64, activation = 'relu'))
model.add(keras.layers.Dense(units = 64, activation = 'relu'))
model.add(keras.layers.Dense(units = 1, activation = 'linear'))
model.compile(loss='mse', optimizer="adam")
model.summary()

In [ ]:
model.fit( x_data, y_data, epochs=100, verbose=1)

In [ ]:
# Compute the output 
y_predicted = model.predict(x_data)

# Display the result
plt.scatter(x_data[::1], y_data[::1])
plt.plot(x_data, y_predicted, 'r', linewidth=4)
plt.grid()
plt.show()